# MGRS project

##### TODO

- Try to obtain similar results as Sofia Antipolis
- Rebuild the analysis from fastq

## 1) Introduction

Laboratory has developped cellular and animal tools to have a better understanding of intrinsic (plasmocyte) and extrinsic (kidney toxicity) impact of some immunoglobulins (Ig).
Part 1 : Some anormal Igs (truncates or incompletes) are toxic for plasmocytes producing it. Factors of this toxicity will be study comparing plasmocytes transcriptome secreting normal vs anormal Ig.
For each condition, 3 duplicates.
Part 2 : We have shown that some Igs could remotly induce kidney disrupt and we want to analyse mechanisms leading to this toxicity. This second part is the 3rd biological replicate of an analysis already start with ReaIg 1922 project and ToxIg2. Analysis done on RNApolyA

## 2) Data

### 2.1) Raw reads

Fastq sent from Sofia-Antipolis

In [ ]:
### CODE ###
# Untar file
tar -xvf sirac.tar;

### 2.2) Mus Musculus

#### 2.2.1) Gencode vs Ensembl

In order to have strong analysis, we manage to get the most comprehensive datasets.
Two consortium deliver annotation genome (Gencode and Ensembl).

We choose for each of them the last release (M16, GRCm38.p5)
Ensembl annotation contains  53946 whereas Gencode annotation contains 53927 genes, which is quite similar.

Using the last transcriptome release of these consortium, we manage to compare annotations with transcritomes against each other to get the better intersection of information.

Gencode transcriptome contains 53379 genes and Gencode annotation contains 53927 genes annotation. 53379 genes are communs, 0 specific to transcriptome and 548 specific to annotation.

Ensembl transcriptome contains 50824 genes and Gencode annotation contains 53927 genes annotation. 50805 genes are communs, 19 specific to transcriptome and 3122 specific to annotation.

Ensembl transcriptome contains 50824 genes and Ensembl annotation contains 53946 genes annotation. 39677 genes are communs, 11110 specific to transcriptome and 26698 specific to annotation.

Gencode transcriptome contains 53379 genes and Ensembl annotation contains 53946 genes annotation. 42232 genes are communs, 11147 specific to transcriptome and 11714 specific to annotation.

In terms of transcriptome annotation and commun annotation the deal Gencode transcritome/Gencode annotation is the best.

#### 2.2.2) p5 version vs p6 version

Some tests were also made to take the best reference genome. 3 consortium deliver reference genome, Genome Reference Consortium (GRC), Gencode and Ensembl.

As Ensembl and Gencode have the same last genome release (p5) and that we choose to keep Gencode transcriptome and gencode annotation, Gencode genome is favoured over Ensembl genome.

GRC last genome release is p6 which is one minor release above Gencode.
To make a choice between them, we asume that the number of chromosome in each reference genome will be sufficient.

We get 239 different chromosomes in GRC version and 107 different chromosomes in Gencode version.

In order to stick the counted read calibration that will be made by featureCounts using the annotation file, we need to find a reference genome which is covered by the annotation file.

For GRC genome reference, 86 chromosome names are commun to genome and annotation, 153 chrosomes names are unique to genome and 0 are unique to annotation.

For Gencode genome reference, 86 chromosome names are commun to genome and annotation, 21 chrosomes names are unique to genome and 0 are unique to annotation.

Then, as uniques chromosome will be remove from reference genome, it doesn't matter which one will be taken. So we took Gencode genome reference to keep the same metric (only Gencode).

#### 2.2.3) Reference genome

###### Downloaded from Gencode, latest realease (M16) :

In [ ]:
### CODE ###
url='ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_mouse/release_M16';
axel -q $url/GRCm38.p5.genome.fa.gz;
unpigz GRCm38.p5.genome.fa.gz;

#### 2.2.4) Reference transcriptome

######  Downloaded from Gencode, latest realease (M16) :

In [ ]:
### CODE ###
url='ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_mouse/release_M16';
axel -q $url/gencode.vM16.transcripts.fa.gz;
axel -q $url/gencode.vM16.lncRNA_transcripts.fa.gz;
axel -q $url/gencode.vM16.pc_transcripts.fa.gz;
cat gencode.vM16.transcripts.fa.gz \
    gencode.vM16.lncRNA_transcripts.fa.gz \
    gencode.vM16.pc_transcripts.fa.gz \
    > gencode.vM16_cdna_ncrna_pc.fasta.gz;
rm gencode.vM16.transcripts.fa.gz gencode.vM16.lncRNA_transcripts.fa.gz gencode.vM16.pc_transcripts.fa.gz;
gunzip gencode.vM16_cdna_ncrna_pc.fasta.gz;

#### 2.2.5) GTF annotations

For future use with featureCounts.

Downloaded from "GENCODE". 

"It contains the comprehensive gene annotation on the reference chromosomes, scaffolds, assembly patches and alternate loci (haplotypes)". 

More precisely, it annotate a lot of gene/transcript biotypes : https://www.gencodegenes.org/gencode_biotypes.html.

In [3]:
### CODE ###
url='ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_mouse/release_M16';
axel -q $ubrl/gencode.vM16.chr_patch_hapl_scaff.annotation.gtf.gz;
unpigz gencode.vM16.chr_patch_hapl_scaff.annotation.gtf.gz;

#### 2.2.6) Subsample reference genome

###### Remove reference genome sequences if they are not in the annotation used for featureCounts

In [ ]:
### CODE ###
python /media/sf_raid/Projects/MGRS/subsample_fa_for_gtf_annotation.py -f /media/sf_raid/Data/MGRS/GRCm38.p5.genome.fa -g /media/sf_raid/Data/MGRS/gencode.vM16.chr_patch_hapl_scaff.annotation.gtf

Content of subsample_fna_for_gtf_annotation.py :

In [ ]:
### CODE ###
import os
import sys
import getopt
import os.path
import shutil
from Bio import SeqIO

### A script allowing change header of fa (.fasta) file with gtf annotation

def usage():
	print('Usage:')
	print('\tpython '+sys.argv[0]+' -f <fa file> -g <gtf file>')
	print('\t\t-f or --fa : fa file')
	print('\t\t-g or --gtf : gtf file')

def main(argv):

	fa_file = ""
	gtf_file = ""
		
	try:
		opts, args = getopt.getopt(sys.argv[1:], 'f:g:', ['fa=', 'gtf=', 'help'])
	except getopt.GetoptError:
		usage()
		sys.exit(2)

	if not opts :
		usage()
		sys.exit(2)
	for opt, arg in opts:
		if opt in ('-h', '--help'):
			usage()
			sys.exit(2)
		elif opt in ('-f', '--fa'):
			fa_file = arg
		elif opt in ('-g', '--gtf'):
			gtf_file = arg
		else:
			usage()
			sys.exit(2)

	if (fa_file[-3:]!=".fa" and fa_file[-4:]!=".fna") or os.path.exists(fa_file)==False:
		print("The fa file is missing, not .fa or .fna !\n")
		usage()
		sys.exit(2)
	if gtf_file[-4:]!=".gtf" or os.path.exists(gtf_file)==False:
		print("The gtf file is missing or not .gtf !\n")
		usage()
		sys.exit(2)
	
	output_subsample = fa_file[:-3]+"_subsampled"+".fa"

	print('\n-----------------------------------------')
	print('fa file : '+fa_file)
	print('GTF file : '+gtf_file)
	print('Output file rename: '+output_subsample)
	print('-----------------------------------------\n')

	list_header_fa=[]
	list_header_gtf=[]
	shared_header=[]

	for record in SeqIO.parse(fa_file, 'fasta'):
		if record.id not in list_header_fa:
			list_header_fa.append(record.id)

	f_gtf = open(gtf_file, "r")
	for line in f_gtf:
		if not line.startswith('#'):
			if line.split('\t')[0] not in list_header_gtf:
				list_header_gtf.append(line.split('\t')[0])

	print("Number of chromosome names in fa : ")
	print(str(len(list_header_fa)))
	print("Number of chromosome names in gtf : ")
	print(str(len(list_header_gtf)))

	count_unique_fa=0
	count_unique_gtf=0
	count_both=0

	for i in list_header_fa:
		if i in list_header_gtf:
			shared_header.append(i)
			count_both+=1
		else:
			count_unique_fa+=1

	for i in list_header_gtf:
		if i not in list_header_fa:
			count_unique_gtf+=1

	print("Number of unique in fa : "+str(count_unique_fa))
	print("Number of unique in gtf : "+str(count_unique_gtf))
	print("Number of shared : "+str(count_both))

	f_output_subsample = open(output_subsample, 'a')

	for record in SeqIO.parse(fa_file, 'fasta'):
		if record.id in shared_header:
			f_output_subsample.write(">"+record.id+"\n")
			f_output_subsample.write(str(record.seq)+"\n")

if __name__ =='__main__':
	main(sys.argv[1:])

As shown above, the number fo chromosome names in the reference genome is 107 whereas annotation one is 86. 86 chromosome names are commun and 21 are specific to reference genome. These lasts were remove from the reference genome. 

###### Check :

In [ ]:
### CODE ###
grep -Pio '^>.*' /media/sf_raid/Data/MGRS/GRCm38.p5.genome.fa | sort -u | wc -l;

Result : 86

#### 2.2.7) Rename reference transcriptome

Gencode transcriptome has a huge identifier for each fasta entry. We will lighten it by keeping only the transcript name access.

In [ ]:
### CODE ###
sed 's/|.*//' gencode.vM16_cdna_ncrna_pc.fasta > gencode.vM16_cdna_ncrna_pc_truc.fasta

#### 2.2.8) Chromosome file for Kallisto

Kallisto's last version allow to pass a chromosome table with chromosome and length of this chromosome in order to directly annotate on genome

In [ ]:
### CODE ###
bioawk -c fastx '{print $name, length($seq)}' <(/media/sf_raid/Data/MGRScat GRCm38.p5.genome_subsampled.fa) > /media/sf_raid/Data/MGRS/chromosomes_GRCm38.p5.genome_subsampled.fa.tsv;

## 3) Raw reads QC & filtering

### 3.1) FastQC

In [ ]:
### CODE ###
fastqc /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2016.runs/161202_sirac_wtr_1003/*
fastqc /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2017.runs/170411_Sirac_wtr_1024/170411_Sirac_wtr_1024_mouse/*
fastqc /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2017.runs/171018_sirac_wtr_1069/*
mkdir /media/sf_raid/Data/MGRS/FastQC
mv /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2016.runs/161202_sirac_wtr_1003/*.zip /media/sf_raid/Data/MGRS/FastQC
mv /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2016.runs/161202_sirac_wtr_1003/*.html /media/sf_raid/Data/MGRS/FastQC
mv /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2017.runs/170411_Sirac_wtr_1024/170411_Sirac_wtr_1024_mouse/*.zip /media/sf_raid/Data/MGRS/FastQC
mv /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2017.runs/170411_Sirac_wtr_1024/170411_Sirac_wtr_1024_mouse/*.html /media/sf_raid/Data/MGRS/FastQC
mv /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2017.runs/171018_sirac_wtr_1069/*.zip /media/sf_raid/Data/MGRS/FastQC
mv /media/sf_raid/Data/MGRS/sirac/000-NEXTSEQ-2017.runs/171018_sirac_wtr_1069/*.html /media/sf_raid/Data/MGRS/FastQC

### 3.2) MultiQC

We compile FastqQC analysis for all the data, to highlight potential run effect.

In [ ]:
### CODE ###
cd /media/sf_raid/Data/MGRS/FastQC
multiqc -o 161202 *S7*.zip *S8*.zip *S9*.zip
multiqc -o 170411 *S10*.zip *S11*.zip *S12*.zip
multiqc -o 171018 *S13*.zip *S14*.zip *S15*.zip

Visualize QC results :

In [ ]:
### CODE ###
cd /media/sf_raid/Data/MGRS/FastQC
chromium-browser 161202/multiqc_report.html;
chromium-browser 170411/multiqc_report.html;
chromium-browser 171018/multiqc_report.html;

Conclusions : Excellent data. Quality mean around 30-35. Nothing to trim.

## 4) Mapping

### 4.1) Build index :

#### 4.1.1) Star :

Using genotoul, STAR-2.5.2b

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -M bastien.herve@unilim.fr
#$ -m bea
#$ -l mem=16G
#$ -l h_vmem=64G
#$ -N star_idx
#$ -o /home/hbastien/work/MGRS/star_idx.o
#$ -e /home/hbastien/work/MGRS/star_idx.e

star='/usr/local/bioinfo/src/STAR/STAR-2.5.2b/bin/Linux_x86_64_static/STAR'

###--sjdbGTFfile specifies the path to the file with annotated transcripts in the standard GTF format. STAR will extract splice junctions from this file and use them to greatly improve accuracy of the mapping. While this is optional, and STAR can be run without annotations, using annotations is highly recommended whenever they are available
###for GFF3 formatted annotations you need to use --sjdbGTFtagExonParentTranscript Parent
###--sjdbOverhang specifies the length of the genomic sequence around the annotated junction to be used in constructing the splice junctions database. Ideally, this length should be equal to the ReadLength-1,

$star --runThreadN 8 --runMode genomeGenerate --genomeDir /home/hbastien/work/MGRS/star_index --genomeFastaFiles /home/hbastien/save/MGRS/GRCm38.p5.genome_subsampled.fa --sjdbGTFfile /home/hbastien/save/MGRS/gencode.vM16.chr_patch_hapl_scaff.annotation.gtf --sjdbOverhang 75;

#### 4.1.2) Kallisto :

Using genotoul, Kallisto 0.44.0

In [ ]:
#### CODE ###
#!/bin/bash
#$ -q workq
#$ -M bastien.herve@unilim.fr
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=120G
#$ -N kallisto_idx
#$ -o /home/hbastien/work/MGRS/kallisto_idx.o
#$ -e /home/hbastien/work/MGRS/kallisto_idx.e

kallisto='/usr/local/bioinfo/src/kallisto/kallisto_linux-v0.44.0/kallisto'

#####Index KALLISTO
$kallisto index --make-unique -i /home/hbastien/work/MGRS/kallisto_index/gencode.vM16_cdna_ncrna_pc_renamed.fasta.idx /home/hbastien/save/MGRS/gencode.vM16_cdna_ncrna_pc_renamed.fasta

### 4.2) Map reads

#### 4.2.1) Star :

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=30G
#$ -N star
#$ -o /home/hbastien/work/MGRS/star.o
#$ -e /home/hbastien/work/MGRS/star.e

star='/usr/local/bioinfo/src/STAR/STAR-2.5.2b/bin/Linux_x86_64_static/STAR'

#####Alignment STAR 161202
for id in 'DH_S8' 'FRA_S7' 'WT_S9';
#for id in 'DH_S8';
do
$star --genomeDir /home/hbastien/work/MGRS/star_index/ --readFilesIn <(gunzip -c /home/hbastien/save/MGRS/raw/161202/$id'_R1_001.fastq.gz') <(gunzip -c /home/hbastien/save/MGRS/raw/161202/$id'_R2_001.fastq.gz') --outFileNamePrefix Experiment_161202_Star$id --outSAMtype BAM SortedByCoordinate
echo $star --genomeDir /home/hbastien/work/MGRS/star_index/ --readFilesIn <(gunzip -c /home/hbastien/save/MGRS/raw/161202/$id'_R1_001.fastq.gz') <(gunzip -c /home/hbastien/save/MGRS/raw/161202/$id'_R2_001.fastq.gz') --outFileNamePrefix Experiment_161202_Star$id --outSAMtype BAM SortedByCoordinate
done

#####Alignment STAR 170411
for id in 'DH_ARN_S12' 'FRA_ARN_S11' 'WT_ARN_S10';
#for id in 'DH_S8';
do
$star --genomeDir /home/hbastien/work/MGRS/star_index/ --readFilesIn <(gunzip -c /home/hbastien/save/MGRS/raw/170411/$id'_R1_001.fastq.gz') <(gunzip -c /home/hbastien/save/MGRS/raw/170411/$id'_R2_001.fastq.gz') --outFileNamePrefix Experiment_170411_Star$id --outSAMtype BAM SortedByCoordinate
echo $star --genomeDir /home/hbastien/work/MGRS/star_index/ --readFilesIn <(gunzip -c /home/hbastien/save/MGRS/raw/170411/$id'_R1_001.fastq.gz') <(gunzip -c /home/hbastien/save/MGRS/raw/170411/$id'_R2_001.fastq.gz') --outFileNamePrefix Experiment_170411_Star$id --outSAMtype BAM SortedByCoordinate
done

#####Alignment STAR 171018
for id in 'GLO_DH_S15' 'GLO_FRA_S14' 'GLO_WT_S13';
#for id in 'DH_S8';
do
$star --genomeDir /home/hbastien/work/MGRS/star_index/ --readFilesIn <(gunzip -c /home/hbastien/save/MGRS/raw/171018/$id'_R1_001.fastq.gz') <(gunzip -c /home/hbastien/save/MGRS/raw/171018/$id'_R2_001.fastq.gz') --outFileNamePrefix Experiment_171018_Star$id --outSAMtype BAM SortedByCoordinate
echo $star --genomeDir /home/hbastien/work/MGRS/star_index/ --readFilesIn <(gunzip -c /home/hbastien/save/MGRS/raw/171018/$id'_R1_001.fastq.gz') <(gunzip -c /home/hbastien/save/MGRS/raw/171018/$id'_R2_001.fastq.gz') --outFileNamePrefix Experiment_171018_Star$id --outSAMtype BAM SortedByCoordinate
done

### 4.3) Index BAM

In [1]:
### CODE ###
for f in *.bam;
do samtools index $f;
done;

### 4.4) RNA-seq alignment QC

Outil permettant de faire des analyses statistiques de la qualité du mapping affectué par le mappeur.

Dans certains programmes utilisés, un bed file à 12 colonnes est requis.
Celui ci a été utilisé : https://sourceforge.net/projects/rseqc/files/BED/Mouse_Mus_musculus/

In [ ]:
### CODE ###
for f in *.bam;
do
echo $
#Résume les statistiques du mapping contenu dans le fichier BAM
bam_stat.py -i $f -q 30 > $f.out.bam_stat_stdoutput.txt
#Calcul la distribution des nucléotides clippés au sein des reads
clipping_profile.py -i $f -o $f.$f.out -q 30 -s "PE" > $f.$f.out.clipping_profile_stdoutput.txt
#Calcul des distributions des deletions au sein des reads
deletion_profile.py -i $f -l 75 -o $f.out -n 1000000 -q 30 > $f.out.deletion_profile_stdoutput.txt
#Calcul de la couverture en reads sur les gènes
geneBody_coverage.py -i $f -r /media/sf_raid/Data/MGRS/mm10_GENCODE_VM11_basic.bed -l 100 -f pdf -o $f.out > $f.out.geneBody_coverage_stdoutput.txt
#Détecte la strand spécificité du jeu de données
infer_experiment.py -i $f -r /media/sf_raid/Data/MGRS/mm10_GENCODE_VM11_basic.bed -s 50000000 -q 30 > $f.out.infer_experiment_stdoutput.txt
#Calcul la taille de l'insert dans le jeu de données
inner_distance.py -i $f -r /media/sf_raid/Data/MGRS/mm10_GENCODE_VM11_basic.bed -o $f.out -k 50000000 -l -250 -u 250 -s 5 -q 30 > $f.out.inner_distance_stdoutput.txt
#Calcul les distributions des insertions de nucléotides
insertion_profile.py -i $f -o $f.out -q 30 -s "PE" > $f.out.insertion_profile_stdoutput.txt
#Détecte les reads splicés
junction_annotation.py -i $f -o $f.out -r /media/sf_raid/Data/MGRS/mm10_GENCODE_VM11_basic.bed -m 50 -q 30 > $f.out.junction_annotation_std$f.output.txt
#Regarde la saturation de la profondeur de séquençage
junction_saturation.py -i $f -o $f.out -r /media/sf_raid/Data/MGRS/mm10_GENCODE_VM11_basic.bed -l 5 -u 100 -s 5 -m 50 -v 1 -q 30 > $f.out.junction_saturation.txt
#Calcul la distribution en mismatch au sein des reads
mismatch_profile.py -i $f -l 75 -o $f.out -n 1000000 -q 30 > $f.out.mismatch_profile_stdoutput.txt
#Regarde la distribution des reads dans les exons, introns...
read_distribution.py -i $f -r /media/sf_raid/Data/MGRS/mm10_GENCODE_VM11_basic.bed > $f.out.read_distribution_stdoutput.txt
#Calcul le taux de duplication des reads
read_duplication.py -i $f -o $f.out -u 500 -q 30 > $f.out.read_duplication_stdoutput.txt
#Calcul le %GC
read_GC.py -i $f -o $f.out -q 30 > $f.out.read_GC_stdoutput.txt
#Calcul le % de chaque hexamer dans le jeu de données
#Attention ici il prend un fasta
#read_hexamer.py read_hexamer.py -i $f -r /media/sf_raid/Data/MGRS/GRCm38.p5.genome_subsampled.fa -g /media/sf_raid/Data/MGRS/gencode.vM16_cdna_ncrna_pc_renamed.fasta
#Regarde la fréquence nucléotidique pour chaque nucléotide
read_NVC.py -i $f -o $f.out -x -q 30 > $f.out.read_NVC_stdoutput.txt
#Calcul le score Phred pour chaque position du read
read_quality.py -i $f -o $f.out -r 1 -q 30 > $f.out.read_quality_stdoutput.txt
#Calcul la taille du fragment de chaque gene
RNA_fragment_size.py -i $f -r /media/sf_raid/Data/MGRS/mm10_GENCODE_VM11_basic.bed -q 30 -n 3 > $f.out.RNA_fragment_size_stdoutput.txt
#Test l'intégrité au niveau transcriptionnel
tin.py -i $f -r /media/sf_raid/Data/MGRS/mm10_GENCODE_VM11_basic.bed -c 10 -n 100 > $f.out.tin_stdoutput.txt
done;

###### Moving files

In [ ]:
### CODE ###
mkdir /media/sf_raid/Data/MGRS/RSeQC
mv *.txt /media/sf_raid/Data/MGRS/RSeQC
mv *.xls /media/sf_raid/Data/MGRS/RSeQC
mv *.r /media/sf_raid/Data/MGRS/RSeQC
mv *.pdf /media/sf_raid/Data/MGRS/RSeQC

## 5) Count

### 5.1) FeatureCounts

In [ ]:
### CODE ###
cd /media/sf_raid/Data/MGRS/Star;
for id in 'Experiment_161202_StarDH_S8Aligned.sortedByCoord.out' 'Experiment_161202_StarFRA_S7Aligned.sortedByCoord.out' 'Experiment_161202_StarWT_S9Aligned.sortedByCoord.out' 'Experiment_170411_StarDH_ARN_S12Aligned.sortedByCoord.out' \
          'Experiment_170411_StarFRA_ARN_S11Aligned.sortedByCoord.out' 'Experiment_170411_StarWT_ARN_S10Aligned.sortedByCoord.out' 'Experiment_171018_StarGLO_DH_S15Aligned.sortedByCoord.out' 'Experiment_171018_StarGLO_FRA_S14Aligned.sortedByCoord.out' \
          'Experiment_171018_StarGLO_WT_S13Aligned.sortedByCoord.out';
do Rscript /media/sf_raid/Projects/MGRS/featureCounts.R $id;
done &>> /media/sf_raid/Data/MGRS/FeatureCounts/featureCounts.verbose;

Content of featureCounts.R :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

args = commandArgs(trailingOnly=TRUE)
in_file = paste(args[1], 'bam', sep = '.')
out_file = paste('/media/sf_raid/Data/MGRS/FeatureCounts/', args[1], '.tsv', sep = '')

library(Rsubread)

fc = featureCounts(files=in_file,
                   annot.ext="/media/sf_raid/Data/MGRS/gencode.vM16.chr_patch_hapl_scaff.annotation.gtf",
                   isGTFAnnotationFile=TRUE,
                   GTF.featureType="exon",
                   GTF.attrType="gene_id",
                   allowMultiOverlap=FALSE,
                   ignoreDup=FALSE,
                   isPairedEnd=TRUE,
                   nthreads=8,
                   minMQS=0,
                   maxMOp=10,
                   checkFragLength=FALSE,
                   useMetaFeatures=TRUE,
                   strandSpecific=2,
                   requireBothEndsMapped=TRUE,
                   countChimericFragments=FALSE,
                   primaryOnly=TRUE)
write.table(fc$counts, out_file, append= T, sep='\t', col.names=NA)

### 5.2) Kallisto

On Genotoul via Kallisto, version 0.44.0 :

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -M bastien.herve@unilim.fr
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=30G
#$ -N kallisto
#$ -o /home/hbastien/work/MGRS/kallisto.o
#$ -e /home/hbastien/work/MGRS/kallisto.e

###-b, --bootstrap-samples=INT  Number of bootstrap samples
###-c, --chromosomes            Tab separated file with chrosome names and lengths
###--rf-stranded            Found this trying both case rf and fr, we got more reads pseudoaligned

kallisto='/usr/local/bioinfo/src/kallisto/kallisto_linux-v0.44.0/kallisto'

#####Alignment KALLISTO 161202
for id in 'DH_S8' 'FRA_S7' 'WT_S9';
do
$kallisto quant -i /home/hbastien/work/MGRS/kallisto_index/gencode.vM16_cdna_ncrna_pc_renamed.fasta.idx -b 100 --genomebam --gtf /home/hbastien/save/MGRS/gencode.vM16.chr_patch_hapl_scaff.annotation.gtf --chromosomes /home/hbastien/save/MGRS/chromosomes_GRCm38.p5.genome_subsampled.fa.tsv --rf-stranded -o kallisto$id /home/hbastien/save/MGRS/raw/161202/$id'_R1_001.fastq.gz' /home/hbastien/save/MGRS/raw/161202/$id'_R2_001.fastq.gz'
done
#####Alignment KALLISTO 170411
for id in 'DH_ARN_S12' 'FRA_ARN_S11' 'WT_ARN_S10';
do
$kallisto quant -i /home/hbastien/work/MGRS/kallisto_index/gencode.vM16_cdna_ncrna_pc_renamed.fasta.idx -b 100 --genomebam --gtf /home/hbastien/save/MGRS/gencode.vM16.chr_patch_hapl_scaff.annotation.gtf --chromosomes /home/hbastien/save/MGRS/chromosomes_GRCm38.p5.genome_subsampled.fa.tsv --rf-stranded -o kallisto$id /home/hbastien/save/MGRS/raw/170411/$id'_R1_001.fastq.gz' /home/hbastien/save/MGRS/raw/170411/$id'_R2_001.fastq.gz'
done
#####Alignment KALLISTO 171018
for id in 'GLO_DH_S15' 'GLO_FRA_S14' 'GLO_WT_S13';
do
$kallisto quant -i /home/hbastien/work/MGRS/kallisto_index/gencode.vM16_cdna_ncrna_pc_renamed.fasta.idx -b 100 --genomebam --gtf /home/hbastien/save/MGRS/gencode.vM16.chr_patch_hapl_scaff.annotation.gtf --chromosomes /home/hbastien/save/MGRS/chromosomes_GRCm38.p5.genome_subsampled.fa.tsv --rf-stranded -o kallisto$id /home/hbastien/save/MGRS/raw/171018/$id'_R1_001.fastq.gz' /home/hbastien/save/MGRS/raw/171018/$id'_R2_001.fastq.gz'
done

### 5.3) Downstream analysis compatibility

Slightly modify featureCounts output to make it compatible with future steps of the analysis. We want our gene names to be the same than those contained in ensembl when retrieved with biomart.

#### 5.3.1) FeatureCounts

Remove spreadsheet hidden ' " ' for downstream compatility with our awk manipulation :

In [ ]:
### CODE ###
cd /media/sf_raid/Data/MGRS/FeatureCounts;
for f in *.tsv;
do sed 's/\"//g' $f > ${f%.*}".temp.tsv";
done;

Use awk sub to convert "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" :

In [ ]:
### CODE ###
for f in *.temp.tsv;
do awk -F '\t' -v OFS='\t' 'NR > 1 {sub(/\.[0-9]*/, "", $1)} 1' $f > ${f%%.*}"_ensembl_gene_id.tsv";
done;
rm *.temp.tsv;

Check that the removal of ".[0-9]?" from the gene id column did not change the number of unique gene ids :

In [ ]:
### CODE ###
for f in *.tsv;
do awk -F '\t' '{print $1}' $f | sort -u | wc -l;
done;

We did show that transforming gene name "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" did not change the number unique gene names.

#### 5.3.2) Kallisto

Check that the removal of ".[0-9]?" from the gene id column did not change the number of unique gene ids :

In [ ]:
### CODE ###
cd /media/sf_raid/Data/MGRS/Kallisto;
for d in ./*;
do awk -F '\t' -v OFS='\t' 'NR > 1 {sub(/\.[0-9]*/, "", $1)} 1' $d/"abundance.tsv" > $d/"abundance_ensembl_gene_id.tsv";
done;

Use awk sub to convert "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" :

In [ ]:
### CODE ###
for d in ./*;
do awk -F '\t' '{print $1}' $d/"abundance.tsv" | sort -u | wc -l;
done;

We did show that transforming gene name "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" did not change the number unique gene names.

## 6) DE analysis